In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
from datetime import datetime, timedelta, date
from dateutil import relativedelta as rd

In [2]:
orders = pd.read_csv('orders_20190822.csv', sep=';', decimal=",")

In [3]:
list(orders.columns)

['id_o', 'user_id', 'price', 'o_date']

In [4]:
orders = orders.rename(columns={'id_o': 'id', 'user_id': 'user', 'o_date': 'date'})
list(orders.columns)

['id', 'user', 'price', 'date']

In [6]:
# Проверяем уникальность заказав
len(orders) == len(orders['id'].unique())

True

In [8]:
# Переводим в фортмат даты и удаяем фейковые заказы
orders['date'] = pd.to_datetime(orders.date)
orders = orders[~(orders['price']<=0)]

In [31]:
# Проверяем подготовленный ДатаФрейм
DATA.dtypes

id_o                int64
user_id             int64
price             float64
o_date     datetime64[ns]
dtype: object

In [23]:
# Вводим переменные и новые классы
period_pred = (datetime(2017, 12, 1), datetime(2017, 12, 31)) #Период для пtimedeltaltaltaltaа
days_lost = timedelta(180) # Количеств дней для того, чтобы считить пользователя потерянным 
CC_pred = 0 # Планируемы оборот
order_price_mean = orders['price'].mean()

In [10]:
FACT = orders[orders['date'].between(*period_pred)]['price'].sum()
FACT

322948401.29999995

In [11]:
#Астивные пользователи, которые не считаются потерянныеми
active_users = orders.groupby('user').filter(lambda o: o['id'].count() >= 3 and 
                                              o['date'].max() > period_pred[0] - days_lost)

In [12]:
#Пользователи которые были активны, но перестали покупать
lost_users = orders.groupby('user').filter(lambda o: o['id'].count() >= 3 and 
                                              o['date'].max() <= period_pred[0] - days_lost)

In [13]:
#Пользователи, которы сделали всего 1 заказ
one_users = orders.groupby('user').filter(lambda o: o['id'].count() == 1)

In [14]:
#Пользователи, которы сделали всего 2 заказа
two_users = orders.groupby('user').filter(lambda o: o['id'].count() == 2)

In [16]:
len(one_users) + len(two_users) + len(active_users) + len(lost_users) == len(orders)

True

In [80]:
# Расчет сколько сколько группа приносит в день и умножаем на кол-во дней
active_users_price = active_users.groupby('date')['price'].sum().mean()*(period_pred[1] - period_pred[0]).days

active_users_price

72815771.55061553

In [105]:
# Коэфцифент сезонности из прошлого урокеа
season_coefs = [0.98050609, 0.87231913, 1.03665563, 1.04224875, 0.90690877,
       0.82487778, 0.77827074, 0.8672703 , 0.88338003, 1.14733963,
       1.31235957, 1.34786358]
season_K  = season_coefs[period_pred[0].month - 1]
season_K

1.34786358

In [39]:
# Прогноз по новым пользователям
one_users_mean_order = one_users['price'].mean() #средний заказ людей, которые покупают впервые
period_fact = [d - rd.relativedelta(months=12) for d in period_pred]

users_ly = one_users[one_users['date'].between(*period_fact)]['id'].count()
users_ly_1 = one_users[one_users['date'].between(*[d - rd.relativedelta(months=1) for d in period_fact])]['id'].count()
users_lm = one_users[one_users['date'].between(*[d - rd.relativedelta(months=1) for d in period_pred])]['id'].count()
k = users_ly / users_ly_1 # Коэффициент проста по факту прошлого года
k

1.1584786833624443

In [41]:
one_users_price = users_lm * k * one_users_mean_order

one_users_price

148496824.29416004

In [50]:
# Рассмотрим пользователей с двумя заказами. Определи средний срок между двумя заками.
# Найдем пользователей которые на самом деле отнотсят к группе людей с одним заказом
# После сделаем прогноз по этим двум группам
left = pd.DataFrame(two_users.groupby('user')['date'].min())
right= pd.DataFrame(two_users.groupby('user')['date'].max())
min_max_dates = left.merge(right, left_on='user', right_on='user')
min_max_dates = min_max_dates.rename(columns={'date_x': 'min', 'date_y': 'max'})

In [52]:
# Реальные пользователи с двумя заказами
two_users_ids = min_max_dates[~(min_max_dates['min'] == min_max_dates['max'])] 

In [53]:
# Остальные пользователи, которые сделали два заказа подряд и отностятся к пользователям с одним заказом
# Сделаем по ним прогноз, как и ранее
one_users_more = len(min_max_dates) - len(two_users_ids)
one_users_price_more = one_users_more * k * one_users_mean_order

one_users_price_more

83104560.93254729

In [54]:
return_days = (two_users_ids['max'] - two_users_ids['min']).mean().days
return_days

145

In [55]:
# Реальные заказа для пользователей с двумя заказми
two_users = two_users[two_users['user'].isin(two_users_ids.index)]

In [56]:
# Расчитаеи вероятность того, что пользователь с одним заказом купит еще раз
p_from_one = len(two_users)/(len(one_users) + one_users_more + len(two_users))
p_from_one

0.15504492277226892

In [68]:
# Рассмотри сколько пользователей с одним заказом ранее купит в прогнозируемом периоде
period_one_fact = [d - timedelta(return_days) for d in period_pred]
two_users_price = one_users[one_users['date'].between(*period_one_fact)]['id'].count() * \
                        p_from_one  * (1 + one_users_more / len(one_users)) * one_users_mean_order

two_users_price

12655907.693455499

In [61]:
# Рассчитаем сколько пользователей из ранее потеряных купили еще раз, и расчитаем вероятность возвращения
period_fact_lm = [d - rd.relativedelta(months=1) for d in period_pred]
lost_users_ly = orders[orders['date'] < period_fact_lm[1]].groupby('user').\
    filter(lambda o: o['id'].count() >= 2 and o['date'].max() <= period_fact_lm[0] - days_lost)

In [64]:
# Количество бывших активных пользователей, которые вернулись
lost_users_returned = orders.loc[orders['date'].between(*period_fact_lm) & \
                                 orders['user'].isin(lost_users_ly['user'].unique())]

In [65]:
p_lost_return = len(lost_users_returned)/len(lost_users_ly)
p_lost_return

0.0003618003443563992

In [66]:
lost_users_price = len(lost_users) * p_lost_return * k * order_price_mean
lost_users_price

212654.20432276005

In [108]:
TO = one_users_price + one_users_price_more + two_users_price + lost_users_price

In [113]:
print('Факт: {:.0f}, Прогноз: {:.0f}. Ошибка {:0.3%}'.format(FACT, TO * season_K, TO * season_K/FACT - 1))

Факт: 322948401, Прогноз: 329512138. Ошибка 2.032%
